In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
women = train_data[train_data["Sex"] =="female"]["Survived"]
sum(women)/len(women)

0.7420382165605095

In [6]:
men = train_data[train_data["Sex"] =="male"]["Survived"]
sum(men)/len(men)

0.18890814558058924

In [7]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data.dropna()[features])
y = train_data.dropna()["Survived"]
X_test = pd.get_dummies(test_data.dropna()[features])

In [8]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB


In [23]:
# preprocess data
from sklearn.compose import ColumnTransformer, make_column_selector
# one-hot encoders
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

preprocess_X = ColumnTransformer(
    [
        (
            "onehot",
            OneHotEncoder(
                # keep as single column if binary, otherwise create one column for _each_ category
                drop="if_binary",
                # error if unseen category is found when calling `predict()`.
                # see documentation for useful alternatives when e.g., some class labels
                # are very infrequent
                # handle_unknown="error",
            ),
            # apply to all non-numeric columns
            # (so if you had categorical encoded with integers,
            # the transformer wouldn't be applied)
            ["Pclass", "Sex", "SibSp", "Parch", "Embarked"],
        )
    ],
            # setting to `True` will put transformer name before each feature
    verbose_feature_names_out=False,
# leave other columns as is
# warning: the default value is "drop", which removes all untransformed columns!
    remainder="passthrough",
)

preprocess_y = LabelEncoder()


In [36]:
X = preprocess_X.fit_transform(train_data.drop(columns=["Survived", "Name", "PassengerId", "Ticket"]))
X_test = preprocess_X.fit_transform(test_data)
y = preprocess_y.fit_transform(train_data["Survived"])

In [37]:
X[1]

array([1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 35.0, 53.1, 'C123'], dtype=object)

In [40]:
train_data.drop(columns=["Survived", "Name", "PassengerId", "Ticket"]).head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
1,1,female,38.0,1,0,71.2833,C85,C
3,1,female,35.0,1,0,53.1000,C123,S
6,1,male,54.0,0,0,51.8625,E46,S
10,3,female,4.0,1,1,16.7000,G6,S
11,1,female,58.0,0,0,26.5500,C103,S


In [39]:
preprocess_X.get_feature_names_out()

array(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_male', 'SibSp_0',
       'SibSp_1', 'SibSp_2', 'SibSp_3', 'Parch_0', 'Parch_1', 'Parch_2',
       'Parch_3', 'Parch_4', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'PassengerId', 'Name', 'Age', 'Ticket', 'Fare', 'Cabin'],
      dtype=object)

In [21]:
pd.DataFrame(data=X, columns=preprocess_X.get_feature_names_out())

ValueError: Shape of passed values is (183, 19), indices imply (183, 20)

In [13]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

ValueError: X has 235 features, but RandomForestClassifier is expecting 453 features as input.